<img src="https://pyjamas.live/img/organizers/pyjamas-name.png"/>


## Geoprocessing with Python

Prof. Martin Christen<br/>
mailto:martin.christen@fhnw.ch<br/>
Twitter: @MartinChristen<br/>
LinkedIn: https://www.linkedin.com/in/martinchristen/

### GitHub

This notebook is available at: https://github.com/martinchristen/pyjamas2020

## 1. Installation of required Modules

(This tutorial requires anaconda, if you don't have it yet, download it here: https://www.anaconda.com/download/ ).

If you try without anaconda: good luck!

**This notebook requires Python 3.7**

### Installing Modules (conda)

Install the main modules using conda, dependencies will be resolved (gdal etc.)

    
    conda create --name pyjamas python=3.7 jupyterlab -y
    conda activate pyjamas
    conda install matplotlib -y
    conda install shapely -y
    conda install fiona -y
    conda install rasterio -y
    conda install geopandas -y
    conda install folium -c conda-forge -y
    conda install descartes

### 2. Introduction to Shapely

https://shapely.readthedocs.io/en/stable/manual.html

![Features](img/Features.png)

Shapely supports the following Features:

* Point
* LineString
* LinearRing          
* Polygon
* MultiLineString
* MultiPoint
* MultiPolygon

Let's construct a Hexagon

In [ ]:
import math

radius = 10
coords = []
for i in range(0,6):
    angle = i*60*math.pi/180
    coords.append((radius * math.cos(angle), radius*math.sin(angle)))
coords
    

Definition for Polygons: first and last point must be same

So let's add it

In [ ]:
# First and last point must be same in polygon
coords.append(coords[0])

In [ ]:
coords

In [ ]:
from shapely.geometry import Polygon

In [ ]:
hexagon = Polygon(coords)

In [ ]:
hexagon

In [ ]:
hexagon.length

In [ ]:
hexagon.area

Let's create another Polygon, this time a triangle

In [ ]:
triangle = Polygon([(-5,0),(0,6),(5,0),(-5,0)])

In [ ]:
triangle  # note: this output is scaled

We can plot the exerior using matplotlib:

In [ ]:
import matplotlib.pyplot as plt

x1,y1 = hexagon.exterior.xy
x2,y2 = triangle.exterior.xy
plt.plot(x1,y1, 'r')
plt.plot(x2,y2, 'b')
plt.axis("equal");

In [ ]:
hexagon.difference(triangle)

We can do:
   * intersection
   * union
   * difference
   * symmetric_difference

Let's move the triangle to test this

In [ ]:
triangle = Polygon([(5,0),(10,6),(15,0),(5,0)])

In [ ]:
hexagon.difference(triangle)

In [ ]:
#intersection
hexagon.intersection(triangle)

In [ ]:
hexagon.union(triangle)

In [ ]:
hexagon.symmetric_difference(triangle)  # "xor"

### Binary operations on shapes:

- **contains** (Returns True if the interior of the object intersects the interior of the other but does not contain it, and the dimension of the intersection is less than the dimension of the one or the other.)
- **intersects** (Returns True if the boundary and interior of the object intersect in any way with those of the other.)
- **witin** (Returns True if the object’s boundary and interior intersect only with the interior of the other (not its boundary or exterior).
- **touches** (Returns True if the objects have at least one point in common and their interiors do not intersect with any part of the other.)
- **crosses** (Returns True if the interior of the object intersects the interior of the other but does not contain it, and the dimension of the intersection is less than the dimension of the one or the other.)
- **equals** (Returns True if the set-theoretic boundary, interior, and exterior of the object coincide with those of the other.)

In [ ]:
triangle.intersects(hexagon)

### Other operations

https://shapely.readthedocs.io/en/stable/manual.html

#### Delaunay triangulation

In [ ]:
from shapely.ops import triangulate

t = triangulate(hexagon, tolerance=0.0, edges=False)
# edges=False: return Polygon (otherwise LineString)

In [ ]:
t

In [ ]:
import matplotlib.pyplot as plt

for i in range(0,len(t)):
    x,y = t[i].exterior.xy
    plt.plot(x,y)

plt.axis("equal");

In [ ]:
import matplotlib.pyplot as plt

x,y = hexagon.exterior.xy
plt.plot(x,y)
plt.axis("equal");

In [ ]:
from shapely.geometry import mapping


geojson = mapping(hexagon)

In [ ]:
geojson

Now load a polygon and draw it on map

## 2. Getting Started with Folium

In [ ]:
import folium

m = folium.Map(location=[47.570333604, 7.59348379], zoom_start=16)
m

### Polygon on Map

Now display this Polygon on the folium map:

In [ ]:
import folium

map_ch = folium.Map(location=[0, 0], zoom_start=3)  
folium.GeoJson(geojson).add_to(map_ch)
map_ch

Geographic Coordinates

![Coords](img/longitude.gif)

## 2. GeoPandas

In [ ]:
import pandas as pd

df = pd.read_csv('data/cities5k.csv', encoding="utf-8", sep=",", header=None, low_memory=False)
df.head(3)

### Removing Columns for Demo...

We really have too many columns, to make everything easier, I just reduce to the most important ones and give some column names.
This is all standard pandas...

In [ ]:
df2 = df[[1,4,5,7,14]]
df2.columns = ["name", "lat", "lng", "type", "population"]
df2.head()

In [ ]:
df2.query("name == 'Paris'")

Remove sections, see https://www.geonames.org/export/codes.html for more details

In [ ]:
df2 = df2[df2.type != 'PPLX']
df2.head()

### Creating a GeoPandas Data Frame

We simply need a geometry column with a shaply geometry...

In [ ]:
import geopandas as gpd
from shapely.geometry import Point

geometry = [Point(pos) for pos in zip(df2['lng'], df2['lat'])]
gdf = gpd.GeoDataFrame(df2, geometry=geometry)

gdf.head()

We can remove the columns "lat" and "lng", it is redundant. 

In [ ]:
gdf = gdf.drop(['lat', 'lng'], axis=1)

In [ ]:
%matplotlib inline
gdf.plot(color='green', markersize=15, figsize=(15,9));

In [ ]:
from shapely.geometry import Point

PyConLT = Point([25.287379, 54.716769])

dist = gdf.distance(PyConLT)

In [ ]:
gdf_new = gdf.copy()
gdf_new["distance"] = dist

In [ ]:
s = gdf_new.sort_values(["distance"], ascending=True)
s.head(10)

In [ ]:
gdf_new = gdf.copy()
lt_mask = gdf_new.within(lithuania)

In [ ]:
lt_cities = gdf_new[lt_mask]

In [ ]:
lt_cities[lt_cities.population > 20000]

### Now display cities on a folium map using markers

In [ ]:
import folium

map_cities = folium.Map(location=[54.716769, 25.287379], zoom_start=7)

def create_marker(row):
    lng = row["geometry"].x
    lat = row["geometry"].y
    name = row["name"]
    population = str(int(row["population"]))
    folium.Marker([lat, lng], popup=f'{name}, population:{population}').add_to(map_cities)
    
lt_cities.apply(create_marker, axis=1)
map_cities


### 3. Live Data & GeoPandas

We're looking at the earthquake data from USGS:
https://earthquake.usgs.gov/earthquakes/feed/v1.0/geojson.php

This data is updated every minute

In [ ]:
import requests

url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_week.geojson"
#url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/significant_month.geojson"
#url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_month.geojson"

data = requests.get(url)
file = open("earthquakes.geojson","wb")
file.write(data.content)
file.close()

In [ ]:
import geopandas as gpd

eq_gdf = gpd.read_file("earthquakes.geojson")
eq_gdf.head()

Let's simplify the output and only take most important rows

In [ ]:
eq = eq_gdf[["time","mag", "place","geometry"]].copy()
eq.head()

Look at the histogramm:

In [ ]:
eq.mag.hist(bins=16);

Timestamps in UTC are not really human readable...
Let's convert them

In [ ]:
from datetime import datetime, timezone

data = []
for row in range(0,len(eq)):
    time = eq.iloc[row].time
    t = str(datetime.fromtimestamp(time/1000.0, timezone.utc))
    data.append(t)
    
eq["time_utc"] = data
eq.head()

In [ ]:
eq = eq.drop(['time'], axis=1)

In [ ]:
eq.plot();

Open Natural Earth Dataset with all Polygons of all countries

In [ ]:
gdfAdmin0 = gpd.read_file("data/ne_10m_admin_0_countries/ne_10m_admin_0_countries.shp", encoding="utf-8")
gdfAdmin0.head()

In [ ]:
countries = gdfAdmin0.plot(figsize=(15,9), color="black")
eq.plot(ax=countries, color="red", markersize=10);

In [ ]:
eq.sort_values(["mag"], ascending=False).head()

### 4. More GeoPandas Fun

In [ ]:
import geopandas as gpd

gdfAdmin0 = gpd.read_file("data/ne_10m_admin_0_countries/ne_10m_admin_0_countries.shp", encoding="utf-8")
gdfAdmin0.head()

ch = gdfAdmin0[gdfAdmin0['NAME'] == "Switzerland"]
ch

In [ ]:
ch.plot();

In [ ]:
import folium

center = [47.570333604, 7.59348379] 
map_ch = folium.Map(center, zoom_start=5)   

folium.GeoJson(ch).add_to(map_ch)

map_ch

In [ ]:
import folium

center = [47.570333604, 7.59348379] 
map = folium.Map(center, zoom_start=5)   

folium.GeoJson(lt,style_function=lambda feature: {
        'fillColor': 'green',   # you can also replace this with functions with feature as argument
        'color': 'black',
        'weight': 2,
        'dashArray': '5, 5'
    }).add_to(map)

map

### 5. Raster Data with rasterio



In [ ]:
import rasterio

dataset = rasterio.open('data/BlueMarble.tif', 'r')

In [ ]:
dataset.name

In [ ]:
dataset.count # number of raster bands, in our case 3 for r,g,b

In [ ]:
dataset.width, dataset.height

In [ ]:
dataset.crs

In [ ]:
dataset.bounds

In [ ]:
dataset.transform  # affine transformation pixel to crs

In [ ]:
dataset.transform * (0, 0)    # Pixel to CRS

In [ ]:
~dataset.transform # inverse affine transformation

In [ ]:
~dataset.transform * (0,0) # CRS to Pixel

In [ ]:
px, py = ~dataset.transform * (7.59348379, 47.570333604) # Our Location to Pixel (lng/lat)
print(px,py)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
r = dataset.read(1)
g = dataset.read(2)
b = dataset.read(3)

In [ ]:
rgb = np.dstack((r,g,b))  # stack r,g,b so we can display it...

In [ ]:
fig, ax = plt.subplots(figsize=(15,9))
ax.imshow(rgb, interpolation='nearest')
ax.plot(px,py, 'ro'); 

# GeoPython Conference 2021

Yes, there will be an online GeoPython 2021. April 22-23, 2021

* Follow @GeoPythonConf on Twitter: http://twitter.com/GeoPythonConf
* Register Interest: https://ti.to/pybasel/geopython-2020
* Talk submission starts soon


<img src="img/geopython2021.jpg" width="50%"/>
